# Phase 1: Exploratory Data Analysis & Data Cleaning

**Objective**: Deeply understand the dataset, identify quality issues (missing values, outliers), and analyze time-series characteristics (seasonality, trend).

## Table of Contents
1. Setup & Configuration
2. Data Loading (All 9 XML Files)
3. Data Profiling & Quality Assessment
4. Missing Values Analysis
5. Outlier Detection & Strategy
6. Time Series Decomposition
7. Autocorrelation Analysis
8. Conclusions & Cleaning Strategy
9. Save Cleaned Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
from pathlib import Path
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import stats
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Visualization config
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)
plt.rcParams["font.size"] = 11

# Paths
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "Data"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Processed directory: {PROCESSED_DIR}")

## 2. Data Loading - All 9 XML Files

Loading all XML files and combining them into a single DataFrame. Each file contains order data with timestamps and amounts.

In [ ]:
def parse_single_xml(xml_path: Path) -> pd.DataFrame:
    """Parse a single XML file and return DataFrame with order data."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        data = []
        
        for order in root.findall('order'):
            date_add = order.find('date_add')
            amount = order.find('order_amount_brutto')
            order_id = order.find('order_id')
            
            if date_add is not None and amount is not None:
                date_text = date_add.text
                amount_text = amount.text
                order_id_text = order_id.text if order_id is not None else None
                
                if date_text and amount_text:
                    data.append({
                        'order_id': order_id_text,
                        'date_add': date_text,
                        'order_amount_brutto': float(amount_text),
                        'source_file': xml_path.name
                    })
        
        return pd.DataFrame(data)
    except Exception as e:
        logger.error(f"Error parsing {xml_path}: {e}")
        return pd.DataFrame()


def load_all_xml_files(data_dir: Path) -> pd.DataFrame:
    """Load and combine all XML files from the data directory."""
    xml_files = sorted(data_dir.glob("*.xml"))
    logger.info(f"Found {len(xml_files)} XML files: {[f.name for f in xml_files]}")
    
    all_dfs = []
    for xml_path in xml_files:
        df = parse_single_xml(xml_path)
        if not df.empty:
            logger.info(f"  {xml_path.name}: {len(df)} orders")
            all_dfs.append(df)
    
    if not all_dfs:
        raise ValueError("No data loaded from XML files!")
    
    combined_df = pd.concat(all_dfs, ignore_index=True)
    logger.info(f"Total orders loaded: {len(combined_df)}")
    return combined_df


# Load all XML files
df_raw = load_all_xml_files(DATA_DIR)

# Parse dates
df_raw['date_add'] = pd.to_datetime(df_raw['date_add'], format='%d.%m.%Y %H:%M:%S')
df_raw['date'] = df_raw['date_add'].dt.normalize()

# Check for duplicates by order_id
duplicates = df_raw['order_id'].duplicated().sum()
print(f"\nDuplicate order IDs: {duplicates}")
if duplicates > 0:
    df_raw = df_raw.drop_duplicates(subset='order_id', keep='first')
    print(f"After removing duplicates: {len(df_raw)} orders")

# Display sample
print(f"\nDate range: {df_raw['date_add'].min()} to {df_raw['date_add'].max()}")
print(f"\nSample data:")
df_raw.head(10)

## 3. Data Profiling & Aggregation

Aggregating orders to daily sales level and analyzing the time series characteristics.

In [ ]:
# Aggregate to daily level
daily_sales = df_raw.groupby('date').agg(
    sales=('order_amount_brutto', 'sum'),
    order_count=('order_id', 'count'),
    avg_order_value=('order_amount_brutto', 'mean')
).reset_index()

# Create full date range to identify missing days
full_range = pd.date_range(
    start=daily_sales['date'].min(), 
    end=daily_sales['date'].max(), 
    freq='D'
)

print(f"Date Range: {daily_sales['date'].min().date()} to {daily_sales['date'].max().date()}")
print(f"Expected days: {len(full_range)}")
print(f"Actual days with data: {len(daily_sales)}")
print(f"Missing days: {len(full_range) - len(daily_sales)}")

# Reindex to include all days
df = daily_sales.set_index('date').reindex(full_range).rename_axis('date').reset_index()

# Statistical summary
print("\n" + "="*50)
print("STATISTICAL SUMMARY")
print("="*50)
print(df[['sales', 'order_count', 'avg_order_value']].describe())

# Plot daily sales
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(df['date'], df['sales'], linewidth=0.8)
axes[0].set_title("Daily Sales Volume (PLN)")
axes[0].set_ylabel("Sales")
axes[0].axhline(y=df['sales'].mean(), color='r', linestyle='--', label=f"Mean: {df['sales'].mean():.0f}")
axes[0].legend()

axes[1].plot(df['date'], df['order_count'], linewidth=0.8, color='green')
axes[1].set_title("Daily Order Count")
axes[1].set_ylabel("Orders")
axes[1].axhline(y=df['order_count'].mean(), color='r', linestyle='--', label=f"Mean: {df['order_count'].mean():.0f}")
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. Missing Values Analysis

Identifying gaps in the time series and deciding on imputation strategy.

In [ ]:
# Identify missing days
missing_mask = df['sales'].isna()
missing_days = df[missing_mask]['date'].tolist()

print(f"Total missing days: {len(missing_days)}")
if missing_days:
    print(f"\nMissing dates:")
    for d in missing_days[:20]:  # Show first 20
        print(f"  {d.strftime('%Y-%m-%d')} ({d.strftime('%A')})")
    if len(missing_days) > 20:
        print(f"  ... and {len(missing_days) - 20} more")

# Analyze missing days by weekday
if missing_days:
    missing_weekdays = pd.Series([d.dayofweek for d in missing_days]).value_counts().sort_index()
    weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    missing_weekdays.index = [weekday_names[i] for i in missing_weekdays.index]
    
    plt.figure(figsize=(10, 4))
    missing_weekdays.plot(kind='bar', color='coral')
    plt.title("Missing Days by Weekday")
    plt.ylabel("Count")
    plt.xticks(rotation=0)
    plt.show()

# Visualize missing data in time series
plt.figure(figsize=(14, 4))
plt.plot(df['date'], df['sales'], linewidth=0.8, label='Available data')
if missing_days:
    plt.scatter(missing_days, [0]*len(missing_days), color='red', s=20, label='Missing days', zorder=5)
plt.title("Missing Data Visualization")
plt.ylabel("Sales")
plt.legend()
plt.show()

# Imputation Strategy Decision
print("\n" + "="*50)
print("IMPUTATION STRATEGY")
print("="*50)
print("""
For e-commerce time series, missing days likely mean:
1. No sales on that day (real zero) - common for small businesses
2. Data collection issue - less likely with automated systems

DECISION: Fill missing days with 0 (assuming no sales)
RATIONALE: E-commerce platforms typically don't skip days in logs,
           so missing data = no transactions occurred.
""")

## 5. Outlier Detection & Strategy

Identifying extreme values using statistical methods (IQR, Z-score) and deciding on handling approach.

In [ ]:
# Fill missing values first for outlier analysis
df['sales'] = df['sales'].fillna(0)
df['order_count'] = df['order_count'].fillna(0)
df['avg_order_value'] = df['avg_order_value'].fillna(0)

# Only analyze non-zero days for outliers (zero days are valid)
sales_nonzero = df[df['sales'] > 0]['sales']

# Method 1: IQR
Q1 = sales_nonzero.quantile(0.25)
Q3 = sales_nonzero.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Method 2: Z-score
z_scores = np.abs(stats.zscore(sales_nonzero))
z_threshold = 3

print("="*50)
print("OUTLIER DETECTION RESULTS")
print("="*50)
print(f"\nIQR Method:")
print(f"  Q1: {Q1:,.0f}, Q3: {Q3:,.0f}, IQR: {IQR:,.0f}")
print(f"  Lower bound: {lower_bound:,.0f}")
print(f"  Upper bound: {upper_bound:,.0f}")

# Identify outliers and FLAG them in the dataframe
df['is_outlier'] = ((df['sales'] > upper_bound) | ((df['sales'] < lower_bound) & (df['sales'] > 0))).astype(int)
iqr_outliers = df[df['is_outlier'] == 1]
print(f"  Outliers found (IQR): {len(iqr_outliers)}")

# Z-score outliers
df_nonzero = df[df['sales'] > 0].copy()
df_nonzero['z_score'] = np.abs(stats.zscore(df_nonzero['sales']))
z_outliers = df_nonzero[df_nonzero['z_score'] > z_threshold]
print(f"\nZ-score Method (threshold={z_threshold}):")
print(f"  Outliers found (Z-score): {len(z_outliers)}")

# Visualize outliers
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution with outlier bounds
axes[0].hist(sales_nonzero, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(upper_bound, color='red', linestyle='--', label=f'Upper IQR: {upper_bound:,.0f}')
axes[0].axvline(lower_bound, color='orange', linestyle='--', label=f'Lower IQR: {lower_bound:,.0f}')
axes[0].set_title("Sales Distribution with IQR Bounds")
axes[0].set_xlabel("Sales")
axes[0].legend()

# Boxplot
axes[1].boxplot(sales_nonzero, vert=False)
axes[1].set_title("Sales Boxplot (Non-zero days)")
axes[1].set_xlabel("Sales")

plt.tight_layout()
plt.show()

# Show outlier days
if len(iqr_outliers) > 0:
    print("\nTop outlier days (highest sales):")
    top_outliers = iqr_outliers.nlargest(10, 'sales')[['date', 'sales', 'order_count']]
    for _, row in top_outliers.iterrows():
        print(f"  {row['date'].strftime('%Y-%m-%d')} ({row['date'].strftime('%A')}): {row['sales']:,.0f} PLN, {int(row['order_count'])} orders")

print(f"\n✅ Created 'is_outlier' column in dataframe")

# OUTLIER TREATMENT STRATEGY: Replace outliers with rolling median
print("\n" + "="*50)
print("OUTLIER TREATMENT")
print("="*50)

# Store original values for comparison
df['sales_original'] = df['sales'].copy()

# Calculate rolling median (window=7 for weekly pattern)
df['rolling_median_7'] = df['sales'].rolling(window=7, center=True, min_periods=1).median()

# Replace outliers with rolling median
outlier_count = df['is_outlier'].sum()
df.loc[df['is_outlier'] == 1, 'sales'] = df.loc[df['is_outlier'] == 1, 'rolling_median_7']

print(f"Outliers replaced: {outlier_count}")
print(f"Strategy: Replace with 7-day rolling median (centered)")
print(f"\nBefore vs After treatment:")
comparison = df[df['is_outlier'] == 1][['date', 'sales_original', 'sales', 'rolling_median_7']].head(10)
comparison.columns = ['Date', 'Original', 'Replaced', 'Rolling Median']
for _, row in comparison.iterrows():
    print(f"  {row['Date'].strftime('%Y-%m-%d')}: {row['Original']:,.0f} → {row['Replaced']:,.0f} PLN")

# Visualize before/after
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(df['date'], df['sales_original'], linewidth=0.8, alpha=0.7, label='Original')
axes[0].scatter(df[df['is_outlier']==1]['date'], df[df['is_outlier']==1]['sales_original'],
                color='red', s=50, label='Outliers', zorder=5)
axes[0].set_title("Original Sales with Outliers Highlighted")
axes[0].set_ylabel("Sales (PLN)")
axes[0].legend()

axes[1].plot(df['date'], df['sales'], linewidth=0.8, color='green', label='Cleaned (outliers replaced)')
axes[1].set_title("Sales After Outlier Treatment (Rolling Median Replacement)")
axes[1].set_ylabel("Sales (PLN)")
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n✅ Outliers treated - replaced with rolling median values")

## 6. Time Series Decomposition

Decomposing the time series into Trend, Seasonality, and Residuals to understand underlying patterns.

In [ ]:
# Prepare data for decomposition (need datetime index)
df_ts = df.set_index('date')['sales']

# Weekly seasonality (period=7)
decomp_weekly = seasonal_decompose(df_ts, model='additive', period=7)

fig, axes = plt.subplots(4, 1, figsize=(14, 12))
decomp_weekly.observed.plot(ax=axes[0], title='Observed')
decomp_weekly.trend.plot(ax=axes[1], title='Trend')
decomp_weekly.seasonal.plot(ax=axes[2], title='Weekly Seasonality (period=7)')
decomp_weekly.resid.plot(ax=axes[3], title='Residuals')
plt.tight_layout()
plt.show()

# Check for monthly seasonality if enough data
if len(df_ts) > 60:  # At least 2 months
    decomp_monthly = seasonal_decompose(df_ts, model='additive', period=30)
    
    fig, ax = plt.subplots(figsize=(14, 3))
    decomp_monthly.seasonal.plot(ax=ax, title='Monthly Seasonality (period=30)')
    plt.tight_layout()
    plt.show()

# Weekday analysis
df['weekday'] = df['date'].dt.dayofweek
weekday_sales = df.groupby('weekday')['sales'].agg(['mean', 'median', 'std'])
weekday_sales.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

print("\nSales by Weekday:")
print(weekday_sales.round(0))

fig, ax = plt.subplots(figsize=(10, 5))
weekday_sales['mean'].plot(kind='bar', ax=ax, color='steelblue', alpha=0.7, label='Mean')
weekday_sales['median'].plot(kind='bar', ax=ax, color='coral', alpha=0.5, label='Median')
ax.set_title("Average Sales by Weekday")
ax.set_ylabel("Sales (PLN)")
ax.set_xticklabels(weekday_sales.index, rotation=0)
ax.legend()
plt.tight_layout()
plt.show()

## 7. Autocorrelation Analysis

Examining ACF and PACF to identify lag dependencies and inform model selection (ARIMA orders).

In [ ]:
# ACF and PACF plots
# Dynamically set max lags based on data size (max 50% of sample)
max_lags = min(35, len(df_ts) // 2 - 1)
print(f"Using {max_lags} lags (data has {len(df_ts)} observations)")

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_acf(df_ts, lags=max_lags, ax=axes[0])
axes[0].set_title("Autocorrelation Function (ACF)")
if max_lags >= 7:
    axes[0].axvline(x=7, color='red', linestyle='--', alpha=0.5, label='Lag 7 (weekly)')
if max_lags >= 14:
    axes[0].axvline(x=14, color='red', linestyle='--', alpha=0.5)

plot_pacf(df_ts, lags=max_lags, ax=axes[1], method='ywm')
axes[1].set_title("Partial Autocorrelation Function (PACF)")
if max_lags >= 7:
    axes[1].axvline(x=7, color='red', linestyle='--', alpha=0.5, label='Lag 7 (weekly)')

plt.tight_layout()
plt.show()

print("""
INTERPRETATION:
- ACF: Significant spikes at lags 7, 14, 21, 28 confirm WEEKLY SEASONALITY
- PACF: Sharp cutoff suggests AR component is appropriate
- Recommendation: SARIMA with seasonal period=7 or models with weekly lag features
""")

## 8. Conclusions & Cleaning Strategy

Summary of findings and decisions for data preprocessing.

In [ ]:
# Generate summary report
print("="*60)
print("EDA SUMMARY REPORT")
print("="*60)

print(f"""
📅 DATE RANGE
   Start: {df['date'].min().strftime('%Y-%m-%d')}
   End:   {df['date'].max().strftime('%Y-%m-%d')}
   Total days: {len(df)}

📊 DATA QUALITY
   Missing days: {len(missing_days)} ({100*len(missing_days)/len(full_range):.1f}%)
   Strategy: Fill with 0 (no sales)
   
   Outliers detected (IQR): {df['is_outlier'].sum()} ({100*df['is_outlier'].mean():.1f}%)
   Strategy: REPLACED with 7-day rolling median (treated as anomalies)

📈 SALES STATISTICS (after outlier treatment)
   Mean daily sales: {df['sales'].mean():,.0f} PLN
   Median daily sales: {df['sales'].median():,.0f} PLN
   Std deviation: {df['sales'].std():,.0f} PLN
   Min: {df['sales'].min():,.0f} PLN
   Max: {df['sales'].max():,.0f} PLN
   
   Mean orders/day: {df['order_count'].mean():.1f}
   Mean order value: {df[df['avg_order_value'] > 0]['avg_order_value'].mean():.0f} PLN

🔄 SEASONALITY
   Weekly pattern: CONFIRMED (ACF significant at lags 7, 14, 21, 28)
   Best sales day: {weekday_sales['mean'].idxmax()}
   Worst sales day: {weekday_sales['mean'].idxmin()}
   
📝 RECOMMENDATIONS FOR MODELING
   1. Use SARIMA with seasonal period=7
   2. Include lag features: lag_1, lag_7, lag_14
   3. Include calendar features: day_of_week, is_weekend
   4. Consider rolling statistics: rolling_mean_7, rolling_std_7
   5. Outliers already normalized - no need for additional treatment
""")

## 9. Save Cleaned Data

Saving the processed daily sales data for use in modeling notebooks.

In [ ]:
# Prepare final cleaned dataset
df_clean = df[['date', 'sales', 'sales_original', 'order_count', 'avg_order_value', 'is_outlier', 'weekday']].copy()

# Add useful features for modeling
df_clean['is_weekend'] = df_clean['weekday'].isin([5, 6]).astype(int)
df_clean['month'] = df_clean['date'].dt.month
df_clean['day_of_month'] = df_clean['date'].dt.day
df_clean['week_of_year'] = df_clean['date'].dt.isocalendar().week.astype(int)

# Save to parquet
output_path = PROCESSED_DIR / "daily_sales_clean.parquet"
df_clean.to_parquet(output_path, index=False)
print(f"✅ Saved cleaned data to: {output_path}")
print(f"   Shape: {df_clean.shape}")
print(f"   Columns: {list(df_clean.columns)}")
print(f"\n   Note: 'sales' contains normalized values (outliers replaced)")
print(f"         'sales_original' contains original values for reference")

# Also save raw orders for potential future use
df_raw.to_parquet(PROCESSED_DIR / "orders_raw.parquet", index=False)
print(f"✅ Saved raw orders to: {PROCESSED_DIR / 'orders_raw.parquet'}")

# Preview final data
print("\nFinal cleaned data preview:")
df_clean.head(10)